In [170]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.models import *
plt.style.use('ggplot')

In [171]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.linear_model import SGDClassifier, PassiveAggressiveClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import plot_roc_curve, confusion_matrix, classification_report

In [172]:
from nltk.tokenize import sent_tokenize
import nltk
import re
# nltk.download()

In [302]:
fake_df = pd.read_csv('data/Fake.csv')
true_df = pd.read_csv('data/True.csv')

In [303]:
fake_df['title'].str.replace(r"\(.*\)","")
fake_df['title'].str.replace('WATCH:', '')

0         Donald Trump Sends Out Embarrassing New Year’...
1         Drunk Bragging Trump Staffer Started Russian ...
2         Sheriff David Clarke Becomes An Internet Joke...
3         Trump Is So Obsessed He Even Has Obama’s Name...
4         Pope Francis Just Called Out Donald Trump Dur...
                               ...                        
23476    McPain: John McCain Furious That Iran Treated ...
23477    JUSTICE? Yahoo Settles E-mail Privacy Class-ac...
23478    Sunnistan: US and Allied ‘Safe Zone’ Plan to T...
23479    How to Blow $700 Million: Al Jazeera America F...
23480    10 U.S. Navy Sailors Held by Iranian Military ...
Name: title, Length: 23481, dtype: object

In [285]:
def remove_organization(x):
    lst = sent_tokenize(x)
    if lst != [] and ' -' in lst[0]:
        lst[0] = lst[0].split(' - ')[1]
    return ''.join(lst)

In [286]:
def clean_titles(x):
    x = x.replace('WATCH:', '')
    if '(' in x:
        x=x.split('(')[0]
    return x

In [287]:
true_df.text = true_df.text.apply(remove_organization)

In [288]:
fake_df.title = fake_df.title.apply(clean_titles)

In [289]:
fake_df['truth'] = 0
true_df['truth'] = 1
all_news_df = pd.concat([fake_df, true_df])

In [294]:
all_news_df.title.iloc[200]

' Bill Maher Takes Down Trump’s Lying Chief Of Staff In BLISTERING Rebuke '

In [295]:
X_train, X_test, y_train, y_test = get_X_y_splits(all_news_df, 'title')

In [296]:
nb_model = naive_bayes_model(X_train, y_train)

In [297]:
nb_model.score(X_test,y_test)

0.9487750556792873

In [306]:
bag = nb_model.best_estimator_.named_steps.vect.get_feature_names()
# counts = nb_model.best_estimator_.named_steps.vect.fit_transform(X_train)
nb_coefs = nb_model.best_estimator_.named_steps.clf.coef_

In [307]:
freq_df = pd.DataFrame(index=bag, data={'coefs': nb_coefs[0]})
freq_df = freq_df.iloc[(-freq_df['coefs'].abs()).argsort()]
freq_df.head(20)

,coefs
00,-13.123277
mode their,-13.123277
mode over,-13.123277
mode on,-13.123277
mode during,-13.123277
mode doesn,-13.123277
mocks what,-13.123277
mocks wh,-13.123277
mocks victims,-13.123277
mocks unprepared,-13.123277
